In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


SPLITTED INTO TRAIN 80%,TEST 10% AND VALIDATION 10%

In [ ]:
import os
import cv2
import numpy as np
from pathlib import Path

# Directories
hand_detected_frames_path = '/content/drive/MyDrive/sign_language_recog/archive/hand_detected_frames'
augmented_frames_path = '/content/drive/MyDrive/sign_language_recog/archive/augmented_frames'

# Create the augmented_frames directory if it doesn't exist
Path(augmented_frames_path).mkdir(parents=True, exist_ok=True)

# Set the target image size for normalization
target_size = (255, 255)  # Define your normalization size here

# Function to rotate an image by a given angle
def rotate_image(image, angle):
    # Get the image dimensions
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)

    # Create a rotation matrix
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

    # Rotate the image
    rotated_image = cv2.warpAffine(image, matrix, (w, h))
    return rotated_image

# Function to process and save only rotated images
def process_and_save_rotated_images(image_path, label, count):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
    img = cv2.resize(img, target_size)  # Resize image to the target size

    # Normalize image values to [0, 1]
    img = img.astype('float32') / 255.0

    # Create label directory if it doesn't exist in augmented_frames
    label_dir = os.path.join(augmented_frames_path, label)
    Path(label_dir).mkdir(parents=True, exist_ok=True)

    # Rotate image by 15 degrees and save as frame1, frame2, etc.
    rotated_img = rotate_image(img, 15)
    rotated_file_path = os.path.join(label_dir, f"frame{count}.jpg")
    cv2.imwrite(rotated_file_path, rotated_img * 255.0)  # Rescale back to 255 for saving

    count += 1  # Increment the frame counter

    return count

# Iterate through each image in hand_detected_frames and process the images
count = 1
for label in os.listdir(hand_detected_frames_path):
    label_path = os.path.join(hand_detected_frames_path, label)

    if os.path.isdir(label_path):
        # Iterate through each image in the label folder
        for image_name in os.listdir(label_path):
            image_path = os.path.join(label_path, image_name)

            if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):  # Only process image files
                count = process_and_save_rotated_images(image_path, label, count)

print(f"Processing completed. {count-1} rotated images have been processed and saved to {augmented_frames_path}.")

Processing completed. 3768 rotated images have been processed and saved to /content/drive/MyDrive/sign_language_recog/archive/augmented_frames.


In [ ]:
import os

# Define the path to the augmented_frames directory
augmented_frames_path = '/content/drive/MyDrive/sign_language_recog/archive/augmented_frames'

# Initialize the label mapping dictionary
label_mapping = {}

# Iterate over each folder in augmented_frames directory (each folder is a label)
for label in os.listdir(augmented_frames_path):
    label_path = os.path.join(augmented_frames_path, label)

    if os.path.isdir(label_path):
        # List all image frames in the label folder
        frame_paths = [os.path.join(label_path, frame) for frame in os.listdir(label_path) if frame.lower().endswith(('.jpg', '.jpeg', '.png'))]

        # Add the label and its frame paths to the dictionary
        label_mapping[label] = frame_paths

# Print the label mapping
for label, paths in label_mapping.items():
    print(f"Label: {label}")
    for path in paths:
        print(f"  Frame Path: {path}")


Label: crazy
  Frame Path: /content/drive/MyDrive/sign_language_recog/archive/augmented_frames/crazy/frame1.jpg
  Frame Path: /content/drive/MyDrive/sign_language_recog/archive/augmented_frames/crazy/frame2.jpg
  Frame Path: /content/drive/MyDrive/sign_language_recog/archive/augmented_frames/crazy/frame3.jpg
  Frame Path: /content/drive/MyDrive/sign_language_recog/archive/augmented_frames/crazy/frame4.jpg
  Frame Path: /content/drive/MyDrive/sign_language_recog/archive/augmented_frames/crazy/frame5.jpg
  Frame Path: /content/drive/MyDrive/sign_language_recog/archive/augmented_frames/crazy/frame6.jpg
  Frame Path: /content/drive/MyDrive/sign_language_recog/archive/augmented_frames/crazy/frame7.jpg
  Frame Path: /content/drive/MyDrive/sign_language_recog/archive/augmented_frames/crazy/frame8.jpg
  Frame Path: /content/drive/MyDrive/sign_language_recog/archive/augmented_frames/crazy/frame9.jpg
  Frame Path: /content/drive/MyDrive/sign_language_recog/archive/augmented_frames/crazy/frame10.

In [ ]:
import os
import shutil
import numpy as np

# Define paths
data_dir = '/content/drive/MyDrive/sign_language_recog/archive/augmented_frames'
train_dir = '/content/drive/MyDrive/sign_language_recog/archive/train'
val_dir = '/content/drive/MyDrive/sign_language_recog/archive/validation'
test_dir = '/content/drive/MyDrive/sign_language_recog/archive/test'

# Create directories for train, validation, and test if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Mapping set names to directories
set_dirs = {
    "train": train_dir,
    "validation": val_dir,
    "test": test_dir
}

# Ensure each label folder exists in both train and validation directories
for label in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label)

    if os.path.isdir(label_dir):  # Process only label folders
        files = os.listdir(label_dir)
        np.random.shuffle(files)

        # Split the files for train, validation, and test sets
        train_split = int(0.8 * len(files))
        val_split = int(0.9 * len(files))

        # Assign files to each set
        sets = {
            "train": files[:train_split],
            "validation": files[train_split:val_split],
            "test": files[val_split:]
        }

        # Copy files to each set without deleting
        for set_name, set_files in sets.items():
            target_dir = os.path.join(set_dirs[set_name], label)
            os.makedirs(target_dir, exist_ok=True)

            for file in set_files:
                # Copy the file to the appropriate directory
                shutil.copy(os.path.join(label_dir, file), os.path.join(target_dir, file))

print("Data splitting and copying completed successfully.")


Data splitting and copying completed successfully.


In [ ]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 65.0 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.0
    Uninstalling tensorflow-2.17.0:
      Successfully uninstalled tensorflow-2.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompat

MAIN PROCESS

BUILDING A MODEL USING CNN AND LSTM WITH 3D CONVULUTION LAYERS.

In [1]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import TimeDistributed, Dense, LSTM, Bidirectional, GlobalAveragePooling2D, Dropout

# Load a pre-trained CNN model to extract features (exclude top layers)
cnn_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Adding a pooling layer on top of the CNN
cnn_output = GlobalAveragePooling2D()(cnn_base.output)
cnn_model = Model(inputs=cnn_base.input, outputs=cnn_output)

# Make CNN model trainable or not depending on your setup
cnn_model.trainable = False  # Set True if fine-tuning

# Define the main model using Sequential
model = Sequential()

# Use TimeDistributed to apply the CNN to each frame in the sequence
model.add(TimeDistributed(cnn_model, input_shape=(None, 224, 224, 3)))

# Add a Bidirectional LSTM layer
model.add(Bidirectional(LSTM
(64, return_sequences=False)))

# Add dropout for regularization
model.add(Dropout(0.5))

# Final dense layer for classification
model.add(Dense(44, activation='softmax'))  # Adjust the output units for your number of classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed (TimeDistributed)   │ (None, None, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 128)                 │         688,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 44)                  │           5,676 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,952,300 (11.26 MB)

 Trainable params: 694,316 (2.65 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [2]:
def __data_generation(self, batch_videos):
    X = np.zeros((self.batch_size, self.sequence_length, *self.image_size, 3), dtype='float32')
    y = np.zeros((self.batch_size, self.num_classes), dtype='float32')

    for i, video in enumerate(batch_videos):
        video_path = os.path.join(self.directory, video)
        frames = sorted([os.path.join(video_path, img) for img in os.listdir(video_path) if img.endswith('.jpg')])

        # Pad or truncate frames to match sequence_length
        if len(frames) < self.sequence_length:
            frames = frames + [frames[-1]] * (self.sequence_length - len(frames))  # Pad with the last frame
        else:
            frames = frames[:self.sequence_length]  # Truncate to the required length

        for j, frame_path in enumerate(frames):
            img = load_img(frame_path, target_size=self.image_size)
            X[i, j] = img_to_array(img) / 255.0

        label = os.path.basename(video_path)
        y[i, self.label_map[label]] = 1

    return X, y


In [3]:
from tensorflow.keras import layers, models

def create_model(image_size, num_classes):
    model = models.Sequential()

    # Apply CNN layers on each frame
    model.add(layers.TimeDistributed(layers.Conv2D(32, (3, 3), activation='relu'), input_shape=(None, *image_size, 3)))
    model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))
    model.add(layers.TimeDistributed(layers.Flatten()))

    # Use Global Average Pooling to handle variable-length sequences
    model.add(layers.GlobalAveragePooling1D())

    # Fully connected layers for classification
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


ALLOWING SEQUENTIAL LENGTH

In [4]:
from tensorflow.keras import layers, models

def create_model(image_size, num_classes):
    model = models.Sequential()

    # Define input shape with an Input layer
    model.add(layers.Input(shape=(None, *image_size, 3)))  # `None` allows variable-length sequences

    # Apply CNN layers on each frame
    model.add(layers.TimeDistributed(layers.Conv2D(32, (3, 3), activation='relu')))
    model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))
    model.add(layers.TimeDistributed(layers.Flatten()))

    # Use Global Average Pooling to handle variable-length sequences
    model.add(layers.GlobalAveragePooling1D())

    # Fully connected layers for classification
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [5]:
import os

# Define your training directory
train_dir = '/content/drive/MyDrive/sign_language_recog/archive/train'

# Create a label mapping based on folder names in the training directory
labels = sorted([d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))])
label_map = {label: idx for idx, label in enumerate(labels)}

# Check the number of classes
num_classes = len(label_map)

print("Label Map:", label_map)
print("Number of Classes:", num_classes)


Label Map: {'a lot': 0, 'arm': 1, 'audience': 2, 'birth': 3, 'contact': 4, 'convince': 5, 'cool': 6, 'corn': 7, 'correct': 8, 'count': 9, 'country': 10, 'cousin': 11, 'crave': 12, 'crazy': 13, 'daily': 14, 'dance': 15, 'dancer': 16, 'dangerous': 17, 'dark': 18, 'date': 19, 'day': 20, 'deaf': 21, 'decorate': 22, 'degree': 23, 'department': 24, 'depressed': 25, 'discuss': 26, 'divide': 27, 'educate': 28, 'eternity': 29, 'german': 30, 'ghost': 31, 'hearing aid': 32, 'lawyer': 33, 'milk': 34, 'monster': 35, 'my': 36, 'necklace': 37, 'odd': 38, 'part': 39, 'peach': 40, 'philosophy': 41, 'procrastinate': 42, 'professor': 43, 'pursue': 44, 'skate': 45, 'sleepy': 46, 'sore throat': 47, 'special': 48, 'study': 49, 'switzerland': 50, 'toilet': 51, 'tornado': 52, 'weight': 53, 'you': 54}
Number of Classes: 55


In [6]:
# Assuming `image_size` and `num_classes` are already defined
image_size = (224, 224)
num_classes = len(label_map)

model = create_model(image_size, num_classes)


In [7]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence

class VideoDataGenerator(Sequence):
    def __init__(self, directory, batch_size, image_size, label_map):
        self.directory = directory
        self.batch_size = batch_size
        self.image_size = image_size
        self.label_map = label_map

        # Gather file paths
        self.file_paths = []
        self.labels = []

        # Populate file_paths and labels
        for label in label_map.keys():
            label_dir = os.path.join(directory, label)
            for file in os.listdir(label_dir):
                if file.endswith('.mp4') or file.endswith('.avi'):  # Adjust for your video format
                    self.file_paths.append(os.path.join(label_dir, file))
                    self.labels.append(label_map[label])

    def __len__(self):
        # Return the number of batches per epoch
        return len(self.file_paths) // self.batch_size

    def __getitem__(self, idx):
        # Get batch file paths and labels
        batch_x = self.file_paths[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx + 1) * self.batch_size]

        # Initialize empty lists for videos and labels
        video_data = []
        labels = []

        for video_path, label in zip(batch_x, batch_y):
            # Load and preprocess each video
            frames = self.load_and_preprocess_video(video_path)
            video_data.append(frames)
            labels.append(label)

        return np.array(video_data), tf.keras.utils.to_categorical(labels, num_classes=len(self.label_map))

    def load_and_preprocess_video(self, video_path):
        # Placeholder for actual video loading and preprocessing
        # Load video frames and resize to `self.image_size`
        frames = []  # Load frames here
        # Resize each frame and process as needed
        return frames


In [8]:
import cv2

class VideoDataGenerator(Sequence):
    def __init__(self, directory, batch_size, image_size, label_map):
        self.directory = directory
        self.batch_size = batch_size
        self.image_size = image_size
        self.label_map = label_map

        # Gather file paths and labels
        self.file_paths = []
        self.labels = []

        for label in label_map.keys():
            label_dir = os.path.join(directory, label)
            for file in os.listdir(label_dir):
                if file.endswith('.mp4') or file.endswith('.avi'):  # Adjust for your video format
                    self.file_paths.append(os.path.join(label_dir, file))
                    self.labels.append(label_map[label])

    def __len__(self):
        # Return the number of batches per epoch
        return len(self.file_paths) // self.batch_size

    def __getitem__(self, idx):
        # Get batch file paths and labels
        batch_x = self.file_paths[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx + 1) * self.batch_size]

        video_data = []
        labels = []

        for video_path, label in zip(batch_x, batch_y):
            frames = self.load_and_preprocess_video(video_path)
            video_data.append(frames)
            labels.append(label)

        return np.array(video_data), tf.keras.utils.to_categorical(labels, num_classes=len(self.label_map))

    def load_and_preprocess_video(self, video_path):
        cap = cv2.VideoCapture(video_path)
        frames = []

        # Read frames from video
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Resize the frame to the target image size
            frame_resized = cv2.resize(frame, self.image_size)

            # Normalize the pixel values if needed (e.g., 0-1)
            frame_normalized = frame_resized.astype('float32') / 255.0

            frames.append(frame_normalized)

        cap.release()

        # Convert list of frames to a NumPy array (shape: (num_frames, height, width, channels))
        return np.array(frames)



In [9]:
# Define your directories
train_dir = '/content/drive/MyDrive/sign_language_recog/archive/train'
val_dir = '/content/drive/MyDrive/sign_language_recog/archive/validation'

# Assuming `image_size` and `label_map` are already defined
train_generator = VideoDataGenerator(
    directory=train_dir,
    batch_size=4,
    image_size=image_size,
    label_map=label_map
)

val_generator = VideoDataGenerator(
    directory=val_dir,
    batch_size=4,
    image_size=image_size,
    label_map=label_map
)


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, LSTM, Dense, Dropout, TimeDistributed, GlobalAveragePooling3D
from tensorflow.keras.layers import Reshape

# Example of a more efficient CNN-LSTM model using Conv3D layers
model = Sequential()

# 3D Convolutional layers to extract features from each frame
model.add(Conv3D(32, (3, 3, 3), activation='relu', input_shape=(None, *image_size, 3)))  # Input shape: (None, height, width, channels)
model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Pooling in 3D
model.add(Dropout(0.3))

model.add(Conv3D(64, (3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.3))

# Global Average Pooling to reduce spatial dimensions
model.add(GlobalAveragePooling3D())  # Reduces height and width dimensions to a single value per feature map

# Reshape the output before passing to LSTM (3D tensor: timesteps, features)
model.add(Reshape((-1, 64)))  # Assuming 64 channels after pooling and average pooling

# LSTM layer for temporal processing
model.add(LSTM(128, return_sequences=False))  # LSTM layer expects 3D input (batch_size, timesteps, features)
model.add(Dropout(0.5))

# Fully connected layers
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_map), activation='softmax'))  # Output layer for classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary to check the layers
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                      │ (None, None, 222, 222, 32)  │           2,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d (MaxPooling3D)         │ (None, None, 111, 111, 32)  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, None, 111, 111, 32)  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_1 (Conv3D)                    │ (None, None, 109, 109, 64)  │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_1 (MaxPooling3D)       │ (None, None, 54, 54, 64)    │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, None, 54, 54, 64)    │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling3d             │ (None, 64)                  │               0 │
│ (GlobalAveragePooling3D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 1, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 128)                 │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 55)                  │           3,575 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 168,631 (658.71 KB)

 Trainable params: 168,631 (658.71 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
train_dir='/content/drive/MyDrive/sign_language_recog/archive/train'
class VideoDataGenerator(Sequence):
    def __init__(self, directory, batch_size, image_size, label_map):
        self.directory = directory
        self.batch_size = batch_size
        self.image_size = image_size
        self.label_map = label_map

        # Gather file paths and labels
        self.file_paths = []
        self.labels = []

        # Populate file_paths and labels
        for label in label_map.keys():
            label_dir = os.path.join(directory, label)
            if not os.path.isdir(label_dir):
                print(f"Warning: Directory {label_dir} does not exist.")
                continue  # Skip if directory doesn't exist
            for file in os.listdir(label_dir):
                if file.endswith('.mp4') or file.endswith('.avi'):  # Adjust for your video format
                    self.file_paths.append(os.path.join(label_dir, file))
                    self.labels.append(label_map[label])

        print(f"Found {len(self.file_paths)} video files in total.")
        print(f"File paths: {self.file_paths[:5]}")  # Print first 5 file paths for inspection

    def __len__(self):
        # Return the number of batches per epoch
        return len(self.file_paths) // self.batch_size


In [12]:

train_directory='/content/drive/MyDrive/sign_language_recog/archive/train'
val_directory='/content/drive/MyDrive/sign_language_recog/archive/validation'
print("Files in train directory:", os.listdir(train_directory))
print("Files in validation directory:", os.listdir(val_directory))


Files in train directory: ['crazy', 'cousin', 'dangerous', 'deaf', 'daily', 'corn', 'convince', 'correct', 'dark', 'date', 'cool', 'dance', 'country', 'decorate', 'count', 'day', 'dancer', 'crave', 'discuss', 'depressed', 'degree', 'divide', 'department', 'my', 'pursue', 'sore throat', 'german', 'eternity', 'procrastinate', 'tornado', 'hearing aid', 'contact', 'professor', 'part', 'study', 'philosophy', 'sleepy', 'birth', 'audience', 'arm', 'you', 'weight', 'odd', 'ghost', 'toilet', 'lawyer', 'milk', 'a lot', 'necklace', 'monster', 'peach', 'skate', 'educate', 'switzerland', 'special']
Files in validation directory: ['crazy', 'cousin', 'dangerous', 'deaf', 'daily', 'corn', 'convince', 'correct', 'dark', 'date', 'cool', 'dance', 'country', 'decorate', 'count', 'day', 'dancer', 'crave', 'discuss', 'depressed', 'degree', 'divide', 'department', 'my', 'pursue', 'sore throat', 'german', 'eternity', 'procrastinate', 'tornado', 'hearing aid', 'contact', 'professor', 'part', 'study', 'philosop

In [13]:

train_directory = '/content/drive/MyDrive/sign_language_recog/archive/videos'

# Make sure the directory for 'a lot' contains videos and has the correct path
test_train_directory = '/content/drive/MyDrive/sign_language_recog/archive/videos/a lot'

# Label map for the 'a lot' class
test_label_map = {'a lot': 0}

# Create the test data generator
test_generator = VideoDataGenerator(test_train_directory, batch_size=4, image_size=(224, 224), label_map=test_label_map)

# Print the number of batches in the generator
print("Number of batches in test generator:", len(test_generator))


Found 0 video files in total.
File paths: []
Number of batches in test generator: 0


In [14]:
import os
test_train_directory='/content/drive/MyDrive/sign_language_recog/archive/videos'
# Check the absolute path to ensure it's correct
absolute_path = os.path.abspath(test_train_directory)
print("Absolute path:", absolute_path)


Absolute path: /content/drive/MyDrive/sign_language_recog/archive/videos


In [15]:
import os

videos_dir = '/content/drive/MyDrive/sign_language_recog/archive/videos'

# List all files in the 'videos' directory to check extensions
video_files = [f for f in os.listdir(videos_dir) if f.endswith(('.mp4', '.avi', '.mov'))]
print("Video files found:", video_files)


Video files found: ['coffee_6.mp4', 'coach_4.mp4', 'coach_5.mp4', 'coat_3.mp4', 'clueless_3.mp4', 'clumsy_4.mp4', 'coat_4.mp4', 'clueless_4.mp4', 'abdomen.mp4', 'abdomen_1.mp4', 'abdomen_2.mp4', 'abdomen_3.mp4', 'abdomen_4.mp4', 'accident_1.mp4', 'accent_1.mp4', 'about_2.mp4', 'about_3.mp4', 'above_1.mp4', 'accent_2.mp4', 'above_2.mp4', 'able.mp4', 'accept_1.mp4', 'accept_2.mp4', 'accent_3.mp4', 'accident_2.mp4', 'accident_3.mp4', 'accept_3.mp4', 'about_4.mp4', 'above_3.mp4', 'about_5.mp4', 'accept_4.mp4', 'accent_4.mp4', 'accept_5.mp4', 'able_1.mp4', 'accept_6.mp4', 'accept_7.mp4', 'able_2.mp4', 'accident_4.mp4', 'about_6.mp4', 'above_4.mp4', 'able_3.mp4', 'able_4.mp4', 'action_1.mp4', 'across.mp4', 'accomplish_1.mp4', 'accident_5.mp4', 'across_1.mp4', 'across_2.mp4', 'accountant_1.mp4', 'act_1.mp4', 'accident_6.mp4', 'action_2.mp4', 'act_2.mp4', 'action_3.mp4', 'across_3.mp4', 'accomplish_2.mp4', 'act_3.mp4', 'accident_7.mp4', 'act_4.mp4', 'accountant_2.mp4', 'accomplish_3.mp4', 'acc

In [16]:
class VideoDataGenerator(Sequence):
    def __init__(self, directory, batch_size, image_size, label_map):
        self.directory = directory
        self.batch_size = batch_size
        self.image_size = image_size
        self.label_map = label_map

        # Gather file paths
        self.file_paths = []
        self.labels = []

        # Loop through each video file in the directory
        for label, idx in label_map.items():
            for file in os.listdir(directory):
                if file.endswith(('.mp4', '.avi', '.mov')):
                    self.file_paths.append(os.path.join(directory, file))
                    self.labels.append(idx)

    def __len__(self):
        return len(self.file_paths) // self.batch_size

    def __getitem__(self, idx):
        batch_x = self.file_paths[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx + 1) * self.batch_size]

        video_data = []
        for video_path in batch_x:
            frames = self.load_and_preprocess_video(video_path)
            video_data.append(frames)

        return np.array(video_data), tf.keras.utils.to_categorical(batch_y, num_classes=len(self.label_map))

    def load_and_preprocess_video(self, video_path):
        frames = []  # Load and preprocess video frames here
        return frames


In [17]:


# Define the parameters for the data generator
batch_size = 16  # Adjust as needed
image_size = (224, 224)  # Image size for each frame
label_map = {
    'a lot': 0, 'arm': 1, 'audience': 2, 'birth': 3, 'contact': 4, 'convince': 5, 'cool': 6,
    'corn': 7, 'correct': 8, 'count': 9, 'country': 10, 'cousin': 11, 'crave': 12, 'crazy': 13,
    'daily': 14, 'dance': 15, 'dancer': 16, 'dangerous': 17, 'dark': 18, 'date': 19, 'day': 20,
    'deaf': 21, 'decorate': 22, 'degree': 23, 'department': 24, 'depressed': 25, 'discuss': 26,
    'divide': 27, 'educate': 28, 'eternity': 29, 'german': 30, 'ghost': 31, 'hearing aid': 32,
    'lawyer': 33, 'milk': 34, 'monster': 35, 'my': 36, 'necklace': 37, 'odd': 38, 'part': 39,
    'peach': 40, 'philosophy': 41, 'procrastinate': 42, 'professor': 43, 'pursue': 44, 'skate': 45,
    'sleepy': 46, 'sore throat': 47, 'special': 48, 'study': 49, 'switzerland': 50, 'toilet': 51,
    'tornado': 52, 'weight': 53, 'you': 54
}

# Define the directories where the video data is stored
train_directory = '/content/drive/MyDrive/sign_language_recog/archive/train'
val_directory = '/content/drive/MyDrive/sign_language_recog/archive/validation'
test_directory = '/content/drive/MyDrive/sign_language_recog/archive/test'

# Create the data generators
train_generator = VideoDataGenerator(train_directory, batch_size, image_size, label_map)
val_generator = VideoDataGenerator(val_directory, batch_size, image_size, label_map)
test_generator = VideoDataGenerator(test_directory, batch_size, image_size, label_map)


In [18]:

import os

video_dir = '/content/drive/MyDrive/sign_language_recog/archive/videos'
files = os.listdir(video_dir)
print(f"Files in directory: {files}")


Files in directory: ['coffee_6.mp4', 'coach_4.mp4', 'coach_5.mp4', 'coat_3.mp4', 'clueless_3.mp4', 'clumsy_4.mp4', 'coat_4.mp4', 'clueless_4.mp4', 'abdomen.mp4', 'abdomen_1.mp4', 'abdomen_2.mp4', 'abdomen_3.mp4', 'abdomen_4.mp4', 'accident_1.mp4', 'accent_1.mp4', 'about_2.mp4', 'about_3.mp4', 'above_1.mp4', 'accent_2.mp4', 'above_2.mp4', 'able.mp4', 'accept_1.mp4', 'accept_2.mp4', 'accent_3.mp4', 'accident_2.mp4', 'accident_3.mp4', 'accept_3.mp4', 'about_4.mp4', 'above_3.mp4', 'about_5.mp4', 'accept_4.mp4', 'accent_4.mp4', 'accept_5.mp4', 'able_1.mp4', 'accept_6.mp4', 'accept_7.mp4', 'able_2.mp4', 'accident_4.mp4', 'about_6.mp4', 'above_4.mp4', 'able_3.mp4', 'able_4.mp4', 'action_1.mp4', 'across.mp4', 'accomplish_1.mp4', 'accident_5.mp4', 'across_1.mp4', 'across_2.mp4', 'accountant_1.mp4', 'act_1.mp4', 'accident_6.mp4', 'action_2.mp4', 'act_2.mp4', 'action_3.mp4', 'across_3.mp4', 'accomplish_2.mp4', 'act_3.mp4', 'accident_7.mp4', 'act_4.mp4', 'accountant_2.mp4', 'accomplish_3.mp4', 'ac

In [19]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence

class VideoDataGenerator(Sequence):
    def __init__(self, directory, batch_size, image_size, label_map, max_frames=50):
        self.directory = directory
        self.batch_size = batch_size
        self.image_size = image_size
        self.label_map = label_map
        self.max_frames = max_frames

        # Gather file paths and labels
        self.file_paths = []
        self.labels = []

        for label, idx in label_map.items():
            for file in os.listdir(directory):
                if file.endswith(('.mp4', '.avi', '.mov')):  # Filter video files
                    self.file_paths.append(os.path.join(directory, file))
                    self.labels.append(idx)

    def __len__(self):
        # Returns the number of batches per epoch
        return len(self.file_paths) // self.batch_size

    def __getitem__(self, idx):
        # Get a batch of video paths and labels
        batch_x = self.file_paths[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx + 1) * self.batch_size]

        # Preprocess the batch
        video_data = []
        for video_path in batch_x:
            frames = self.load_and_preprocess_video(video_path)
            video_data.append(frames)

        # Convert list to a 4D numpy array: (batch_size, max_frames, height, width, channels)
        video_data = np.array(video_data)

        # Return the processed batch and one-hot encoded labels
        return video_data, tf.keras.utils.to_categorical(batch_y, num_classes=len(self.label_map))

    def load_and_preprocess_video(self, video_path):
        # Open the video
        cap = cv2.VideoCapture(video_path)
        frames = []

        # Read the frames
        while(cap.isOpened()):
            ret, frame = cap.read()
            if not ret:
                break
            # Resize and normalize frames
            frame = cv2.resize(frame, self.image_size)  # Resize frames to image_size
            frame = frame / 255.0  # Normalize pixel values
            frames.append(frame)

        cap.release()

        # Pad or truncate the frames to the max_frames length
        num_frames = len(frames)
        if num_frames < self.max_frames:
            # Pad the frames with zeros
            frames.extend([np.zeros_like(frames[0])] * (self.max_frames - num_frames))
        elif num_frames > self.max_frames:
            # Truncate the frames
            frames = frames[:self.max_frames]

        return np.array(frames)



32 SAMPLES FROM EVERY TEST CLASSES

In [20]:
# Testing the data generator by checking the output of a batch
train_generator = VideoDataGenerator('/content/drive/MyDrive/sign_language_recog/archive/videos', batch_size=32, image_size=(224, 224), label_map=label_map)

x_batch, y_batch = train_generator.__getitem__(0)
print(f'Batch x shape: {x_batch.shape}')
print(f'Batch y shape: {y_batch.shape}')


Batch x shape: (32, 50, 224, 224, 3)
Batch y shape: (32, 55)


In [21]:
import tensorflow as tf
class VideoDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, video_directory, batch_size, image_size, label_map, **kwargs):
        self.video_directory = video_directory
        self.batch_size = batch_size
        self.image_size = image_size
        self.label_map = label_map
        self.video_paths = self._get_video_paths()  # This method fetches video paths
        self.on_epoch_end()  # Initialize any internal tracking
        super().__init__(**kwargs)  # Ensure to call the parent constructor with any extra arguments


In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize ImageDataGenerator for training and validation
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Set up the train and validation generators
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/sign_language_recog/archive/train',  # Replace with the actual path to your training data
    target_size=(224, 224),  # Adjust size according to model input requirements
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/sign_language_recog/archive/validation',  # Replace with the actual path to your validation data
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 3613 images belonging to 55 classes.
Found 709 images belonging to 55 classes.


MODEL LEARNING

In [23]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(55, activation='softmax')
])


In [24]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Use a suitable loss function for your task
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30
)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


113/113 ━━━━━━━━━━━━━━━━━━━━ 3087s 26s/step - accuracy: 0.0242 - loss: 3.9996 - val_accuracy: 0.0268 - val_loss: 3.9593
Epoch 2/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 216s 1s/step - accuracy: 0.0264 - loss: 3.9641 - val_accuracy: 0.0254 - val_loss: 3.9413
Epoch 3/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step - accuracy: 0.0308 - loss: 3.9377 - val_accuracy: 0.0635 - val_loss: 3.9021
Epoch 4/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 217s 2s/step - accuracy: 0.0643 - loss: 3.8811 - val_accuracy: 0.0973 - val_loss: 3.7751
Epoch 5/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 190s 1s/step - accuracy: 0.0816 - loss: 3.7195 - val_accuracy: 0.0733 - val_loss: 3.5214
Epoch 6/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 164s 1s/step - accuracy: 0.0870 - loss: 3.4640 - val_accuracy: 0.1340 - val_loss: 3.2760
Epoch 7/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 160s 1s/step - accuracy: 0.1240 - loss: 3.2450 - val_accuracy: 0.1523 - val_loss: 3.1050
Epoch 8/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 205s 1s/step - accuracy: 0.1371 - loss: 3.0821 - val_accuracy: 0.1

In [25]:
# Save the model in .keras format
model.save("my_model.keras")


In [26]:
from tensorflow.keras.models import load_model

# Load the previously saved model
model = load_model("my_model.keras")


In [27]:
from tensorflow.keras.optimizers import Adam

# Recompile the model with a new learning rate (if required)
model.compile(optimizer=Adam(learning_rate=0.001),  # Set the learning rate
              loss='categorical_crossentropy',   # Choose the loss function for your task
              metrics=['accuracy'])             # Define the metrics


In [28]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Set up the learning rate reduction callback (if needed)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.5,
                                  patience=5,
                                  verbose=1,
                                  min_lr=1e-6)


In [29]:
history = model.fit(
    train_generator,
    validation_data=val_generator,  # Correct variable name
    epochs=60,
    initial_epoch=30,
    callbacks=[lr_scheduler]  # Ensure lr_scheduler is defined
)


Epoch 31/60
113/113 ━━━━━━━━━━━━━━━━━━━━ 155s 1s/step - accuracy: 0.3506 - loss: 1.9709 - val_accuracy: 0.3822 - val_loss: 1.9198 - learning_rate: 0.0010
Epoch 32/60
113/113 ━━━━━━━━━━━━━━━━━━━━ 154s 1s/step - accuracy: 0.3537 - loss: 1.9286 - val_accuracy: 0.4175 - val_loss: 1.9065 - learning_rate: 0.0010
Epoch 33/60
113/113 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step - accuracy: 0.3789 - loss: 1.9003 - val_accuracy: 0.3907 - val_loss: 1.8648 - learning_rate: 0.0010
Epoch 34/60
113/113 ━━━━━━━━━━━━━━━━━━━━ 166s 1s/step - accuracy: 0.3685 - loss: 1.8562 - val_accuracy: 0.4090 - val_loss: 1.8362 - learning_rate: 0.0010
Epoch 35/60
113/113 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step - accuracy: 0.4083 - loss: 1.8260 - val_accuracy: 0.4274 - val_loss: 1.8116 - learning_rate: 0.0010
Epoch 36/60
113/113 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.4036 - loss: 1.8115 - val_accuracy: 0.3992 - val_loss: 1.7807 - learning_rate: 0.0010
Epoch 37/60
113/113 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.4047 - l

In [30]:
model.save("my_model_60_epochs.keras")


FINAL MODEL WITH 160 EPOCHS

In [31]:
from tensorflow.keras.models import load_model

# Load the model trained up to 60 epochs
model = load_model("my_model_60_epochs.keras")

# Continue training from epoch 61
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=160,                 # Total epochs you want to reach, e.g., 80
    initial_epoch=60           # Resume from epoch 61
)

Epoch 61/160
113/113 ━━━━━━━━━━━━━━━━━━━━ 159s 1s/step - accuracy: 0.5278 - loss: 1.3867 - val_accuracy: 0.4922 - val_loss: 1.3941
Epoch 62/160
113/113 ━━━━━━━━━━━━━━━━━━━━ 197s 1s/step - accuracy: 0.5293 - loss: 1.3655 - val_accuracy: 0.5628 - val_loss: 1.3458
Epoch 63/160
113/113 ━━━━━━━━━━━━━━━━━━━━ 221s 2s/step - accuracy: 0.5044 - loss: 1.3752 - val_accuracy: 0.5092 - val_loss: 1.3460
Epoch 64/160
113/113 ━━━━━━━━━━━━━━━━━━━━ 183s 1s/step - accuracy: 0.5408 - loss: 1.3344 - val_accuracy: 0.5092 - val_loss: 1.3534
Epoch 65/160
113/113 ━━━━━━━━━━━━━━━━━━━━ 152s 1s/step - accuracy: 0.5306 - loss: 1.3443 - val_accuracy: 0.5388 - val_loss: 1.3299
Epoch 66/160
113/113 ━━━━━━━━━━━━━━━━━━━━ 202s 1s/step - accuracy: 0.5349 - loss: 1.3183 - val_accuracy: 0.5360 - val_loss: 1.3464
Epoch 67/160
113/113 ━━━━━━━━━━━━━━━━━━━━ 160s 1s/step - accuracy: 0.5364 - loss: 1.3234 - val_accuracy: 0.5783 - val_loss: 1.3079
Epoch 68/160
113/113 ━━━━━━━━━━━━━━━━━━━━ 155s 1s/step - accuracy: 0.5471 - loss: 1

In [40]:
model.save("final_model_160.keras")

In [39]:
from tensorflow.keras.models import load_model

# Load the final model
model = load_model("final_model_160.keras")


FEW LIBRARIES TO VIEW VIDEO AND DO PREDICTIONS BY TAKING A VIDEO

In [36]:
!apt-get update
!apt-get install -y ffmpeg


Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,108 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,482 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,164 kB]
Get:13 http://archive.ubuntu.com/u

In [37]:
!ffmpeg -version


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l

In [38]:
# Check model summary
model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)                    │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 32)                  │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 128)                 │           4,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 55)                  │           7,095 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 36,647 (143.16 KB)

 Trainable params: 12,215 (47.71 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 24,432 (95.44 KB)

frames of a word:AUDIENCE

In [ ]:
import os

# List files in the directory
os.listdir('/content/drive/MyDrive/sign_language_recog/archive/validation/audience')


['frame2607.jpg',
 'frame2620.jpg',
 'frame2623.jpg',
 'frame2614.jpg',
 'frame2635.jpg',
 'frame2613.jpg']

In [ ]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 253, 253, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 126, 126, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 124, 124, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 115200)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      14,745,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 55)                  │           7,095 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 29,692,144 (113.27 MB)

 Trainable params: 14,846,071 (56.63 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 14,846,073 (56.63 MB)

In [ ]:
print("Model input shape:", model.input_shape)


Model input shape: (None, 255, 255, 3)


ALL FRAMES TO A FRAMEARRAY

In [1]:
from keras.models import load_model
import os
from keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Load the trained model (replace the path with the actual model file path)
model = load_model('/content/final_model_160.keras')

# Path to the directory containing subfolders (each subfolder is a video)
base_path = '/content/drive/MyDrive/sign_language_recog/archive/train/'

# List subfolders (video names)
subfolders = os.listdir(base_path)

# Prepare an array to hold all the frames from all videos
frames_array = []

# Iterate through each subfolder
for subfolder in subfolders:
    subfolder_path = os.path.join(base_path, subfolder)

    # Skip non-directory files (if any)
    if not os.path.isdir(subfolder_path):
        continue

    # List all files (frames) in the subfolder
    frame_files = os.listdir(subfolder_path)

    # Process each frame in the subfolder
    for frame_file in frame_files:
        frame_path = os.path.join(subfolder_path, frame_file)

        # Check if the file is an image (you can filter based on extensions)
        if frame_file.endswith(('.jpg', '.jpeg', '.png')):
            # Load and resize the image to the input size required by the model
            frame = load_img(frame_path, target_size=(255, 255))  # Adjust size to match your model's input size
            frame_array = img_to_array(frame)  # Convert to numpy array
            frames_array.append(np.expand_dims(frame_array, axis=0))  # Add to the list

# Convert list of frames to numpy array for model prediction
frames_array = np.vstack(frames_array)

# Now you can pass frames_array to the model for prediction
predictions = model.predict(frames_array)

# Display the predictions (e.g., the predicted class labels for each frame)
print(predictions)


113/113 ━━━━━━━━━━━━━━━━━━━━ 63s 544ms/step
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


prediction of all 55 classes frames

In [ ]:
import numpy as np

# Assuming predictions is the array you showed above, which has shape (num_frames, num_classes)
# Get the index of the class with the highest probability for each frame
predicted_class_indices = np.argmax(predictions, axis=1)

# Assuming you have a label_map that maps indices to class labels
# Example: label_map = ['label1', 'label2', ..., 'labelN']
label_map = ['a lot', 'arm', 'audience', 'birth', 'contact', 'convince', 'cool', 'corn',
             'correct', 'count', 'country', 'cousin', 'crave', 'crazy', 'daily', 'dance',
             'dancer', 'dangerous', 'dark', 'date', 'day', 'deaf', 'decorate', 'degree',
             'department', 'depressed', 'discuss', 'divide', 'educate', 'eternity', 'german',
             'ghost', 'hearing aid', 'lawyer', 'milk', 'monster', 'my', 'necklace', 'odd',
             'part', 'peach', 'philosophy', 'procrastinate', 'professor', 'pursue', 'skate',
             'sleepy', 'sore throat', 'special', 'study', 'switzerland', 'toilet', 'tornado',
             'weight', 'you']

# Map the predicted class indices to the corresponding class labels
predicted_labels = [label_map[index] for index in predicted_class_indices]

# Now you can print or use the predicted labels
for i, label in enumerate(predicted_labels):
    print(f"Frame {i+1}: Predicted label: {label}")


Frame 1: Predicted label: crazy
Frame 2: Predicted label: crazy
Frame 3: Predicted label: crazy
Frame 4: Predicted label: crazy
Frame 5: Predicted label: crazy
Frame 6: Predicted label: crazy
Frame 7: Predicted label: crazy
Frame 8: Predicted label: crazy
Frame 9: Predicted label: crazy
Frame 10: Predicted label: crazy
Frame 11: Predicted label: crazy
Frame 12: Predicted label: crazy
Frame 13: Predicted label: crazy
Frame 14: Predicted label: crazy
Frame 15: Predicted label: crazy
Frame 16: Predicted label: crazy
Frame 17: Predicted label: crazy
Frame 18: Predicted label: crazy
Frame 19: Predicted label: crazy
Frame 20: Predicted label: crazy
Frame 21: Predicted label: crazy
Frame 22: Predicted label: crazy
Frame 23: Predicted label: crazy
Frame 24: Predicted label: crazy
Frame 25: Predicted label: crazy
Frame 26: Predicted label: crazy
Frame 27: Predicted label: crazy
Frame 28: Predicted label: crazy
Frame 29: Predicted label: crazy
Frame 30: Predicted label: crazy
Frame 31: Predicted